# MUC17 Mutation Trans Effect on Transcriptomics

### Library Imports

In [1]:
import pandas as pd
import numpy as np
import scipy.stats

import warnings
warnings.filterwarnings("ignore")

import cptac
import cptac.utils as al

brain = cptac.Gbm()

### Select Gene

In [2]:
gene = "MUC17"

### Investigate Proteomics, Phosphoproteomics, Acetylproteomics, or Transcriptomics

In [3]:
#omics = "proteomics"
omics = "transcriptomics"
#omics = "phosphoproteomics"
#omics = "acetylproteomics"

### Track all significant comparisons in Dataframe

In [4]:
all_significant_comparisons = pd.DataFrame(columns=['Cancer_Type', 'Gene', 'Comparison','Interacting_Protein','P_Value'])

In [5]:
def add_to_all_significant_comparisons(df, cancer, interacting, all_sig_comp):
    expanded = df
    expanded['Gene'] = gene
    expanded['Cancer_Type'] = cancer
    expanded['Interacting_Protein'] = interacting
    
    updated_all_comparisons = pd.concat([all_sig_comp, expanded], sort=False)
    
    return updated_all_comparisons

# Interacting Proteins: Transcriptomics

### Generate interacting protein list

Make a call to cptac.utils to get the interacting proteins method, which interacts with the uniprot and string databases to generate a list of known interacting partners with the given gene.

In [6]:
# Use get interacting proteins method to generate list of interacting proteins
interacting_proteins = al.get_interacting_proteins(gene)

# Show interacting protein list
print("Interacting Proteins:")
for interacting_protein in interacting_proteins:
    print(interacting_protein)

Interacting Proteins:
GALNT10
ST6GALNAC2
ST3GAL3
MUC17
B3GNT3
ST6GALNAC3
ST6GALNAC4
GALNT12
MUC21
MUC3A
ST3GAL2
GCNT3
MUC16
C1GALT1
MUC6
MUC7
MUC20
MUC15
MUC4
ST3GAL4
GALNT4
MUC5B
MUC12
MUC1
MUC13
MUC5AC


### Test for significant comparisons in any of interacting proteins

In [7]:
# Create dataframe in order to do comparisons with wrap_ttest
protdf = brain.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins)
protdf = brain.reduce_multiindex(protdf, flatten=True)
protdf = protdf[protdf.Sample_Status == 'Tumor'] # drop Normal samples

# Create the binary valued column needed to do the comparison
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

# Format the dataframe correctly
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)


# Make list of columns to be compared using t-tests
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

# Call wrap_ttest, pass in formatted dataframe
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

# Print results, if anything significant was found
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")
        
        all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Gbm", True, all_significant_comparisons)
        

Doing t-test comparisons

No significant comparisons.


# All Proteins: Transcriptomics 

In [8]:
try:
    print("\nGene: ", gene)

    # Use all proteins

    # Create dataframe in order to do comparisons with wrap_ttest
    protdf = brain.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics)
    protdf = brain.reduce_multiindex(protdf, flatten=True) # flatten for specific column names
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
    
    # Create binary column 
    protdf['Label'] = np.where(
                protdf[gene+'_Mutation_Status'] == 'Wildtype_Tumor', 'Wildtype', 'Mutated')

    # Format the dataframe correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    # Make list of columns to be compared using t-tests
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")
    
    # Call wrap_ttest, pass in formatted dataframe
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    # Print results, if anything significant was found
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")
            
            all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Gbm", False, all_significant_comparisons)


except Exception as e:
    print("Error in Comparison")
    print(e)


Gene:  MUC17


Doing t-test comparisons

                                         Comparison       P_Value
0         SNORA70_transcriptomics_ENSG00000207098.1  1.350109e-09
1       LINC00664_transcriptomics_ENSG00000268658.4  6.607980e-09
2    RP11-777B9.4_transcriptomics_ENSG00000248128.1  1.661603e-08
3          CALCA_transcriptomics_ENSG00000110680.11  2.709938e-08
4   CTD-2384A14.1_transcriptomics_ENSG00000258038.4  1.896927e-07
5           CD177_transcriptomics_ENSG00000204936.8  3.184732e-07
6           Y_RNA_transcriptomics_ENSG00000252266.1  3.383993e-07
7        MTND2P14_transcriptomics_ENSG00000240796.1  3.800779e-07
8          PRSS58_transcriptomics_ENSG00000258223.5  4.154030e-07
9     RP5-827O9.1_transcriptomics_ENSG00000229201.1  4.368703e-07
10     AC079163.1_transcriptomics_ENSG00000226218.1  5.852272e-07
11        OR52B3P_transcriptomics_ENSG00000175800.5  7.316744e-07





# Print all significant comparisons

In [9]:
if len(all_significant_comparisons) > 0:
    display(all_significant_comparisons)
    
else:
    print('No Significant Comparisons!')

,Cancer_Type,Gene,Comparison,Interacting_Protein,P_Value
0,Gbm,MUC17,SNORA70_transcriptomics_ENSG00000207098.1,False,1.350109e-09
1,Gbm,MUC17,LINC00664_transcriptomics_ENSG00000268658.4,False,6.607980e-09
2,Gbm,MUC17,RP11-777B9.4_transcriptomics_ENSG00000248128.1,False,1.661603e-08
3,Gbm,MUC17,CALCA_transcriptomics_ENSG00000110680.11,False,2.709938e-08
4,Gbm,MUC17,CTD-2384A14.1_transcriptomics_ENSG00000258038.4,False,1.896927e-07
5,Gbm,MUC17,CD177_transcriptomics_ENSG00000204936.8,False,3.184732e-07
6,Gbm,MUC17,Y_RNA_transcriptomics_ENSG00000252266.1,False,3.383993e-07
7,Gbm,MUC17,MTND2P14_transcriptomics_ENSG00000240796.1,False,3.800779e-07
8,Gbm,MUC17,PRSS58_transcriptomics_ENSG00000258223.5,False,4.154030e-07
9,Gbm,MUC17,RP5-827O9.1_transcriptomics_ENSG00000229201.1,False,4.368703e-07


### Write Significant Comparisons (if any) to Shared CSV file

In [10]:
'''
existing_results = pd.read_csv(gene+'_Trans_Results.csv')

updated_results = pd.concat([existing_results, all_significant_comparisons], sort=False)

updated_results.to_csv(path_or_buf = gene + '_Trans_Results.csv', index=False)'''

"\nexisting_results = pd.read_csv(gene+'_Trans_Results.csv')\n\nupdated_results = pd.concat([existing_results, all_significant_comparisons], sort=False)\n\nupdated_results.to_csv(path_or_buf = gene + '_Trans_Results.csv', index=False)"